In [2]:
import pinecone
import numpy as np
import pandas as pd

e:\Anaconda\envs\pinecone\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
sample = pd.read_csv('../data/sample.csv')
sample

,text
0,The future king is the prince
1,Daughter is the princess
2,Son is the prince
3,Only a man can be a king
4,Only a woman can be a queen
5,The princess will be a queen
6,Queen and king rule the realm
7,The prince is a strong man
8,The princess is a beautiful woman
9,The royal family is the king and queen and the...


In [21]:
df = pd.read_csv("../data/embedding_dict.csv")
df

,keys,values
0,beautiful,"[0.9838497638702393, -1.6587189435958862]"
1,boy,"[-1.5598138570785522, -0.11537443846464157]"
2,can,"[-0.429669052362442, 0.8653393983840942]"
3,children,"[1.0019017457962036, -0.02745703235268593]"
4,daughter,"[1.2124338150024414, -1.6305664777755737]"
5,family,"[0.9203447103500366, 1.5682507753372192]"
6,future,"[-0.9258182644844055, 0.3344442844390869]"
7,king,"[0.6992722749710083, -0.06783338636159897]"
8,man,"[-0.801942765712738, 0.4273762106895447]"
9,now,"[-1.0517592430114746, 1.4044568538665771]"


In [33]:
import numpy as np
import pandas as pd

rows = df['values']
embeddings = []
for j in rows:
    list1 = j
    num1 = ''
    pos1 = False
    flag = True
    num2 = ''
    pos1 = False
    temp = df['values'].str.strip('[]')
    for i in list1:
        if i == '[' or i == ']':
            continue
        if i == ',':
           flag = False
           continue
        if flag:
           if i == '-':
               pos1=True
               continue
           num1 = num1+i
        else:
           if i == '-':
               pos2=True
               continue
           num2 = num2+i
    num1 = pd.to_numeric(num1)
    num2 = pd.to_numeric(num2)
    if pos1:
        num1 *= -1
    if pos2:
        num2 *= -1
    vector = [num1, num2]
    embeddings.append(vector)
    print(vector)

[0.9838497638702391, -1.6587189435958862]
[-1.5598138570785522, -0.1153744384646415]
[-0.429669052362442, -0.8653393983840942]
[1.0019017457962036, -0.0274570323526859]
[1.2124338150024414, -1.6305664777755735]
[0.9203447103500366, -1.5682507753372192]
[-0.9258182644844056, -0.3344442844390869]
[0.6992722749710083, -0.0678333863615989]
[-0.801942765712738, -0.4273762106895447]
[-1.0517592430114746, -1.4044568538665771]
[-1.1261903047561646, -1.1733978986740112]
[-0.0148269431665539, -1.2010029554367063]
[0.7874786257743835, -1.1477224826812744]
[0.6614691615104675, -0.4165874123573303]
[0.2802084684371948, -1.200094223022461]
[-0.0239750631153583, -0.8417088985443115]
[1.0308167934417725, -0.9422696232795716]
[-0.8190408945083618, -0.8789239525794983]
[-1.0741701126098633, -0.4627572298049927]
[0.6947928071022034, -0.9626272320747375]
[0.3819427788257599, -0.904999017715454]


In [49]:
embeddings = np.array(embeddings).astype(float)

In [13]:
from pinecone import Pinecone, ServerlessSpec
from tqdm.autonotebook import tqdm

pc = Pinecone(api_key="508a1fea-8fd8-4b51-ae51-053df59dd9a7")

In [36]:
pc.create_index(
    "demo", 
    dimension=2, 
    metric="dotproduct", 
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)
# initialize index
index = pc.Index("demo")

In [37]:
index.describe_index_stats()

{'dimension': 2,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [50]:
index.upsert(
    vectors = zip(df['keys'], embeddings),
    namespace='ns3'
)

{'upserted_count': 21}

In [52]:
index.query(
    namespace='ns3',
    vector=[-1.5598138570785522, -0.11537443846464157],
    top_k = 10,

)

{'matches': [{'id': 'boy', 'score': 2.44633031, 'values': []},
             {'id': 'only', 'score': 1.89202738, 'values': []},
             {'id': 'now', 'score': 1.80258703, 'values': []},
             {'id': 'strong', 'score': 1.72889578, 'values': []},
             {'id': 'future', 'score': 1.48269057, 'values': []},
             {'id': 'son', 'score': 1.37895668, 'values': []},
             {'id': 'man', 'score': 1.30018973, 'values': []},
             {'id': 'can', 'score': 0.770041823, 'values': []},
             {'id': 'prince', 'score': 0.161692321, 'values': []},
             {'id': 'royal', 'score': 0.134508327, 'values': []}],
 'namespace': 'ns3',
 'usage': {'read_units': 5}}